<a href="https://colab.research.google.com/github/UmarKhattab09/NLP/blob/main/BiDirectional_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fake News Using Bi-Directional LSTM

In [1]:
import pandas as pd
df = pd.read_csv('train.csv')

In [2]:
df.head()

,label,news
0,False,Says the Annies List political group supports ...
1,True,When did the decline of coal start? It started...
2,True,"Hillary Clinton agrees with John McCain ""by vo..."
3,False,Health care reform legislation is likely to ma...
4,True,The economic turnaround started at the end of ...


In [3]:
df.isna().sum()

,0
label,0
news,0


In [5]:
df.shape

(10240, 2)

In [6]:
X = df.drop('label',axis = 1)
y = df['label']

In [7]:
y.value_counts()

,count
label,
True,5752
False,4488


In [8]:
import tensorflow as tf

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [21]:
##VocabularySize
VocabularySize=5000

In [10]:
messages = X.copy()

In [12]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:

### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['news'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
corpus[1]

'declin coal start start natur ga took start begin presid georg w bush administr'

In [18]:
messages['news'][1]

'When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.'

In [22]:
onehot_repr=[one_hot(words,VocabularySize)for words in corpus]

In [26]:
onehot_repr[1]

[3046,
 4781,
 1724,
 1724,
 3676,
 913,
 264,
 1724,
 3214,
 197,
 3493,
 1726,
 3871,
 2614]

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1850  222 1099 ...    0    0    0]
 [3046 4781 1724 ...    0    0    0]
 [3656 2828 4389 ...    0    0    0]
 ...
 [1850 4060 2040 ...    0    0    0]
 [2877 3352 1218 ...    0    0    0]
 [4400 1420 2445 ...    0    0    0]]


In [28]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(VocabularySize,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) ### BIDIRECTIONAL
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [37]:
### Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 14s 79ms/step - accuracy: 0.5554 - loss: 0.6865 - val_accuracy: 0.6009 - val_loss: 0.6617
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.6664 - loss: 0.6210 - val_accuracy: 0.5970 - val_loss: 0.6698
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.7211 - loss: 0.5555 - val_accuracy: 0.5805 - val_loss: 0.7048
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - accuracy: 0.7649 - loss: 0.4872 - val_accuracy: 0.5704 - val_loss: 0.8245
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 75ms/step - accuracy: 0.8139 - loss: 0.4103 - val_accuracy: 0.5645 - val_loss: 0.9329
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 9s 63ms/step - accuracy: 0.8539 - loss: 0.3435 - val_accuracy: 0.5586 - val_loss: 0.9727
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.8688 - loss: 0.3037 - val_accuracy: 0.5473 - val_loss: 1.3201
Epoch 8/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 11s 71ms/step - accuracy: 0.8891 - loss: 0.2591 - val

In [42]:
y_pred = model.predict(X_test)


106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [44]:
y_pred = np.where(y_pred>0.5,1,0)
y_pred

array([[1],
       [0],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [45]:
y_test,y_pred

(array([ True, False, False, ...,  True,  True, False]),
 array([[1],
        [0],
        [1],
        ...,
        [1],
        [1],
        [0]]))

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[ 710,  746],
       [ 764, 1160]])